# Few shots classsification using SetFit

In [1]:
!pip install setfit
# !pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, wh

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

import torch
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import Dataset, DatasetDict, load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from sentence_transformers.losses import CosineSimilarityLoss


2024-01-30 19:26:39.107795: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 19:26:39.107930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 19:26:39.238717: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# wandb login enabled by default in SetFit, if installed
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_token = user_secrets.get_secret("wandb_key") 
wandb.login(key=wandb_token)

/opt/conda/lib/python3.10/site-packages/wandb/sdk/launch/builder/build.py:11: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_names

True

## Load data

Custom dataset previously cleaned, split and saved to HF Hub

In [4]:
filepath = "gentilrenard/lmd_ukraine_comments"

ds = load_dataset(filepath)
train_dataset = ds['train']
eval_dataset = ds['validation']

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Generating train split:   0%|          | 0/323 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/139 [00:00<?, ? examples/s]

Generating unlabeled split:   0%|          | 0/174891 [00:00<?, ? examples/s]

## Modeling, using *Sklearn LogisticRegression* head

Note : our own tests and also by the authors, LogisticRegression gives better results than a differentiable, torch head.  
Model, classification head type (rforest, GBM...) and params, hyperparameters were chosen after multiple experiments.  
See hyperparameters optimization notebook.

In [5]:
# Optional : sample dataset, X number of examples per class
# train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=80, seed=40)

In [6]:
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    head_params={
        "solver": "liblinear",
        "max_iter": 137
    }
)

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [7]:
args = TrainingArguments(
    batch_size=32,
    body_learning_rate=3e-7,
    num_epochs=2,
    max_steps=1950,  # 2350 + load best model True
    sampling_strategy='oversampling',
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps = 50,
    save_total_limit = 1,
    report_to = 'wandb',
    run_name = 'setfit_optimized_v5',
    load_best_model_at_end=False,
)

In [8]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
)

Map:   0%|          | 0/323 [00:00<?, ? examples/s]

In [9]:
trainer.train()

***** Running training *****
  Num unique pairs = 68504
  Batch size = 32
  Num epochs = 2
  Total optimization steps = 1950
wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Currently logged in as: vionmatthieu. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240130_192711-9jleyk3i
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run setfit_optimized_v5
wandb: ⭐️ View project at https://wandb.ai/vionmatthieu/setfit
wandb: 🚀 View run at https://wandb.ai/vionmatthieu/setfit/runs/9jleyk3i


Step,Training Loss,Validation Loss,Embedding Loss,Rate
50,No log,No log,0.263500,0.000000
100,No log,No log,0.260500,0.000000
150,No log,No log,0.256100,0.000000
200,No log,No log,0.253100,0.000000
250,No log,No log,0.249300,0.000000
300,No log,No log,0.246400,0.000000
350,No log,No log,0.244500,0.000000
400,No log,No log,0.241800,0.000000
450,No log,No log,0.240500,0.000000
500,No log,No log,0.238200,0.000000


  0%|          | 0/397 [00:00<?, ?it/s]

  0%|          | 0/397 [00:00<?, ?it/s]

  0%|          | 0/397 [00:00<?, ?it/s]

...

  0%|          | 0/397 [00:00<?, ?it/s]

  0%|          | 0/397 [00:00<?, ?it/s]

  0%|          | 0/397 [00:00<?, ?it/s]

  0%|          | 0/397 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

## Evaluate

In [10]:
metrics = trainer.evaluate(eval_dataset)
print(metrics)

***** Running evaluation *****
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


{'accuracy': 0.7482014388489209}


## Export model to huggingface hub

In [11]:
filepath_model = "gentilrenard/paraphrase-multilingual-mpnet-base-v2_setfit-lemonde-french-v2"

In [12]:
# optional push model to the hub

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_key") 

trainer.push_to_hub(filepath_model, use_auth_token=hf_token)

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


model_head.pkl:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gentilrenard/paraphrase-multilingual-mpnet-base-v2_setfit-lemonde-french-v2/commit/21030a9577de026238872a4e76495a9016edb3a1', commit_message='Add SetFit model', commit_description='', oid='21030a9577de026238872a4e76495a9016edb3a1', pr_url=None, pr_revision=None, pr_num=None)

## Load from hub / inference

In [13]:
# Download from Hub
model = SetFitModel.from_pretrained(filepath_model)

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

config_setfit.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

In [14]:
# Run inference
preds = model.predict(
    [
        "La Russie va gagner cette guerre, ils ont plus de ressources",
        "les journalistes sont corrompus, le traitement est partial",
        "les pauvres ukrainiens se font anéantir et subissent des crimes de guerre",
        "La France doit donner plus d'armes à l'ukraine"
    ]
)
print(preds)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([1, 2, 0, 0])


## Save model to local dir

In [15]:
# save to local directory
save_directory = "/kaggle/working/" 
trainer.model._save_pretrained(save_directory=save_directory)

In [16]:
# load local file and predict
model = SetFitModel.from_pretrained(save_directory)
preds = model.predict(
    [
        "La Russie va gagner cette guerre, ils ont plus de ressources",
        "les journalistes sont corrompus, le traitement est partial",
        "les pauvres ukrainiens se font anéantir et subissent des crimes de guerre",
        "La France doit donner plus d'armes à l'ukraine"
    ]
)
print(preds)

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([1, 2, 0, 0])
